# yFiles Juypter Graphs for SPARQL v1.0 <a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs-for-sparql/blob/main/examples/releases/v1.0.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook introduces the features of the initial release of `yfiles-jupyter-graphs-for-sparql`, an open-source descendant of [yfiles-jupyter-graphs](https://github.com/yWorks/yfiles-jupyter-graphs). t is specifically designed for RDF databases, allowing you to easily visualize the results of your SPARQL queries using the powerful graph visualization SDK [yFiles](https://www.yfiles.com/).

The interactive visualization features, including various automatic layouts, a neighborhood view, a data view, and a search function, help you gain new insights from your query results. Combined with powerful data mappings, you can effortlessly create compelling visual representations of your RDF database.

In [ ]:
%pip install SPARQLWrapper --quiet
%pip install yfiles_jupyter_graphs_for_sparql --quiet
from SPARQLWrapper import SPARQLWrapper, JSON

from yfiles_jupyter_graphs_for_sparql import SparqlGraphWidget

You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

# Connect to RDF database

The connection to the database is handled by the connector you pass. The initial release expects a `SPARQLWrapper` instance but the widget's open-source design is such that it could also handle different connectors.

In [ ]:
# create a connector
connector = SPARQLWrapper("https://dbpedia.org/sparql")

# pass it to the SparqlGraphWidget
widget = SparqlGraphWidget(wrapper=connector)

# Visualize the database schema

In [ ]:
widget.show_schema()

# Query data with SPARQL

The configured widget can then be used to query the RDF database with SPARQL for some data.

In the first release, we require the result to be `SELECT` triples of subject, predicate, object or `CONSTRUCT` query results to visualize graph results.

There are plans to support any `SELECT` query result in the future. For example by querying the local neighborhood of single subject/objects to create an actual graph result. The implicitly queried data could then be visualized differently (e.g. greyed out) to bring the explicitly queried data in the focus.

In [ ]:
query = """SELECT DISTINCT ?sub ?p ?obj
WHERE {
  VALUES ?film {
    <http://dbpedia.org/resource/Pulp_Fiction>
    <http://dbpedia.org/resource/The_Avengers_(2012_film)>
    <http://dbpedia.org/resource/Django_Unchained>
  }
  
  {
    ?film ?p ?obj .
    BIND(?film AS ?sub)
    FILTER(?p IN (dbo:director, dbo:starring, dbo:producer, dbo:cinematography, dbo:musicComposer))
  }
}
LIMIT 40 """

# visualizing queries requires the JSON return format
connector.setReturnFormat(JSON)
# visualize the result as graph
widget.show_query(query)

# Data-driven visualization mappings

Similar to [yfiles-jupyter-graphs](https://github.com/yWorks/yfiles-jupyter-graphs), this widget provides methods to map data properties to visual features.

In [ ]:
# actors
widget.add_object_configuration('starring', color='purple')
# movies
widget.add_subject_configuration('director', color='blue')
# producer and director edges
widget.add_predicate_configuration(['producer', 'director'], color='orange')
# all edges, where no specific configuration is given
widget.add_predicate_configuration('*', thickness_factor= 2) 

widget.show_query(query)

# Automatic layouts

The powerful automatic layouts by [yFiles](https://www.yfiles.com/) provide arrangements for man different use cases. Each highlighting different aspects of the graph.

The widget either allows to interactively change the layout with the "Layout" button in the toolbar, or by providing a specific layout with the query.

In [ ]:
widget.show_query(query, layout='orthogonal')

# Visualize numerical values as heat

Often data represents stress or capacity. This can be visualized explicitly with a heat mapping. In this case, we will use random heat values to illustrate this feature.

In [ ]:
import random
def random_heat(item):
    return max(0.3, (min(random.random(), 0.8)))

widget.add_object_configuration('starring', color='purple', heat=random_heat)
widget.show_query(query, layout='circular')

# Visualize relationships as grouped hierarchy

The widget supports to visualize specific predicates as parent-child relationships. This creates a more natural hierarchy for some relationships and introduces an easier understandable structure to the result graph.

In [ ]:
widget.add_parent_configuration('starring', True)
widget.add_parent_configuration('director')
widget.show_query(query, layout='hierarchic')